In [2]:
import pandas as pd

In [3]:
intake24_df = pd.read_csv('files/intake24_survey_file.csv')
print(intake24_df)

                                 Survey ID  User ID                Start time  \
0     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
1     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
2     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
3     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
4     5d3e72da-a241-4e8e-837b-021accf9a37c        1  2022-08-03T06:55:02.102Z   
...                                    ...      ...                       ...   
5749  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5750  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5751  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5752  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   
5753  4f7e1c91-c883-438a-b1b5-43bf6f0222cc      263  2022-09-23T11:34:28.178Z   

               Submission t

# Breakdown (From user to ingredient):

The file has many users.

Each user has many surveys.

Each survey has many meals.

Each meal consists of many food components.

Every food component is marked with a "Nutrition ID code".

In [18]:
def rename_columns(old_name:str, new_name: str, df: pd.DataFrame) -> None:
    df.rename(columns = { old_name : new_name }, inplace=True)
    return None

rename_columns('Survey ID', 'survey_id', intake24_df)
rename_columns('Meal ID', 'meal_id', intake24_df)

In [16]:
# Structure is like this:
# - Map user ID to their respective meal and information
# - Map the meal number to the respective ingredients
# - Map the ingredients to their respective nutrient code

user_information = {}

In [20]:
def loop_meals(meal_ids_list: list, survey_meals_df: pd.DataFrame) -> None:

    for meal_id in meal_ids_list:
        print(meal_id)
    ...

In [21]:
# Step 1: Let's split by the survey ID.

# Get the list of survey IDs.
survey_ids_list = intake24_df['survey_id'].unique().tolist()
print(f'Total number of surveys carried out: {len(survey_ids_list)}')

# Loop one survey ID at a time and split the Intake24 dataframe accordingly.
for id in survey_ids_list:

    survey_meals_df = intake24_df.query("survey_id == @id")

    # Step 2: Split further the Meal ID
    meal_ids_list = survey_meals_df['meal_id'].unique().tolist()
    loop_meals(meal_ids_list, survey_meals_df)
    break

Total number of surveys carried out: 405
1
2
3
4
